In [1]:
import os
# set device GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
import time
from tensorflow import keras
from tensorflow.keras import layers

tf.enable_eager_execution()
tf.test.is_gpu_available()

import sys
sys.path.append('../')

from config import *

from tools import *

import warnings

warnings.filterwarnings('ignore')
tf.debugging.set_log_device_placement(True)

### 载入统计数据

In [4]:
stat_age_tr = pd.read_pickle(TRAIN_DIR+'feat-impor-age-value.pkl')
stat_age_ts = pd.read_pickle(TEST_DIR+'feat-impor-age-value.pkl')

In [5]:
stat_gender_tr = pd.read_pickle(TRAIN_DIR+'feat-impor-gender-value.pkl')
stat_gender_ts = pd.read_pickle(TEST_DIR+'feat-impor-gender-value.pkl')

In [6]:
cols = list(set(stat_age_tr.columns) & set(stat_gender_tr.columns))

In [7]:
stat_gender_tr = stat_gender_tr.drop(cols, axis=1)
stat_gender_ts = stat_gender_ts.drop(cols, axis=1)

In [8]:
stat_tr = pd.concat([stat_age_tr, stat_gender_tr], axis=1)
stat_ts = pd.concat([stat_age_ts, stat_gender_ts], axis=1)

In [9]:
tr_user_log = pd.read_pickle(TRAIN_DIR+USER_LOG_PATH).groupby(['user_id']).agg({'age': 'first', 'gender': 'first'}).reset_index()

tr_user_log['age'] = tr_user_log['age'] - 1
tr_user_log['gender'] = tr_user_log['gender'] - 1

In [10]:
tr_df = pd.concat([stat_tr, tr_user_log[['age', 'gender']]], axis=1)
ts_df = stat_ts

### 切分 train 和 test 数据集

In [11]:
msk = np.random.rand(len(tr_df)) <= 0.8
vl_df = tr_df[~msk]
tr_df = tr_df[msk]

### 配置超参数

In [38]:
EPOCHS = 1
BATCH_SIZE = 1024
BUFFER_SIZE = 1024

### 生成dataset

In [24]:
### make train dataset
age = tr_df.pop('age')
gender = tr_df.pop('gender')

tr_ds = tf.data.Dataset.from_tensor_slices((tr_df.values, (age.values, gender.values)))

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [25]:
### make valid dataset
age = vl_df.pop('age')
gender = vl_df.pop('gender')

vl_ds = tf.data.Dataset.from_tensor_slices((vl_df.values, (age.values, gender.values)))

In [26]:
### make test dataset
ts_ds = tf.data.Dataset.from_tensor_slices(ts_df.values)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [27]:
tr_ds = tr_ds.cache()
tr_ds = tr_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
tr_ds = tr_ds.prefetch(tf.data.experimental.AUTOTUNE)

vl_ds = vl_ds.cache()
vl_ds = vl_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
vl_ds = vl_ds.prefetch(tf.data.experimental.AUTOTUNE)

Executing op CacheDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [28]:
tf.reset_default_graph()

### 优化器和学习率

In [31]:
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

### 创建模型

In [32]:
class MyModel(tf.keras.Model):
    def __init__(self,
                 dim0=256, dim1=64, dim2=32, num_class1=10, num_class2=2):
        super(MyModel, self).__init__()
        
        self.dense0 = tf.keras.layers.Dense(dim0, activation='relu', name='dense0')
        self.dense1 = tf.keras.layers.Dense(dim1, activation='relu', name='dense1')
        self.dense2 = tf.keras.layers.Dense(dim2, activation='relu', name='dense2')
        self.dense3_age = tf.keras.layers.Dense(num_class1, activation='softmax', name='softmax1')
        self.dense_gender = tf.keras.layers.Dense(num_class2, activation='softmax', name='softmax2')

    def call(self, x):
        x = self.dense0(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3_age(x), self.dense_gender(x)

In [33]:
model = MyModel()

### 损失函数和metric

In [34]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

def loss_function(real, pred):
    pred_age = pred[0]
    real_age = real[0]

    pred_gender = pred[1]
    real_gender = real[1]

    loss1 = loss_object(real_age, pred_age)
    loss2 = loss_object(real_gender, pred_gender)

    loss_ = 0.5*loss1 + 0.5*loss2
    return tf.reduce_mean(loss_)

### train metric
train_loss = tf.keras.metrics.Mean(name='train_loss')
age_train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
gender_train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

### test metric
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
age_valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')
gender_valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


### 开始训练

In [35]:
@tf.function
def train_step(inp, tar):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = loss_function(tar, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    age_train_accuracy(tar[0], predictions[0])
    gender_train_accuracy(tar[1], predictions[1])

@tf.function
def valid_step(inp, tar):
    predictions = model(inp)
    loss = loss_function(tar, predictions)

    valid_loss(loss)
    age_valid_accuracy(tar[0], predictions[0])
    gender_valid_accuracy(tar[1], predictions[1])

In [42]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    age_train_accuracy.reset_states()
    gender_train_accuracy.reset_states()
    
    valid_loss.reset_states()
    age_valid_accuracy.reset_states()
    gender_valid_accuracy.reset_states()    

    '''
    inp1: ad_id list;
    inp2: product_id list;
    inp3: advertiser_id list;
    inp4: click_times list;
    '''
    for (batch, (inp, tar)) in enumerate(tr_ds):
        train_step(inp, tar)

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Age-Accuracy {:.4f} Gender-Accuracy {:.4f} Time taken for training: {} secs'
                   .format(epoch + 1, batch, train_loss.result(), 
                           age_train_accuracy.result(), gender_train_accuracy.result(),
                          time.time()-start))
            start = time.time()
            
    tmp = time.time()
    for inp, tar in vl_ds:
        valid_step(inp, tar)    
    print('########################################## valid ################################################')
    print('Epoch {} Batch {} Loss {:.4f} Age-Accuracy {:.4f} Gender-Accuracy {:.4f} \
          Time taken for validation: {} secs'
          .format(epoch + 1, batch, 
                  valid_loss.result(), age_valid_accuracy.result(), gender_valid_accuracy.result(),
                  time.time() - tmp))

Epoch 1 Batch 0 Loss 0.8041 Age-Accuracy 0.4434 Gender-Accuracy 0.9238 Time taken for training: 2.017078399658203 secs
Epoch 1 Batch 100 Loss 0.7791 Age-Accuracy 0.4410 Gender-Accuracy 0.9290 Time taken for training: 0.7714321613311768 secs
Epoch 1 Batch 200 Loss 0.7801 Age-Accuracy 0.4414 Gender-Accuracy 0.9288 Time taken for training: 0.7405295372009277 secs
Epoch 1 Batch 300 Loss 0.7797 Age-Accuracy 0.4415 Gender-Accuracy 0.9289 Time taken for training: 0.7571663856506348 secs
Epoch 1 Batch 400 Loss 0.7794 Age-Accuracy 0.4413 Gender-Accuracy 0.9290 Time taken for training: 0.7492825984954834 secs
Epoch 1 Batch 500 Loss 0.7792 Age-Accuracy 0.4416 Gender-Accuracy 0.9290 Time taken for training: 0.7617392539978027 secs
Epoch 1 Batch 600 Loss 0.7787 Age-Accuracy 0.4419 Gender-Accuracy 0.9289 Time taken for training: 0.7578902244567871 secs
Epoch 1 Batch 700 Loss 0.7783 Age-Accuracy 0.4422 Gender-Accuracy 0.9291 Time taken for training: 0.7782814502716064 secs
###########################